In [22]:
!pip install ortools
#!pip install dtale

In [23]:
import pandas as pd
import numpy as np
import collections
from ortools.sat.python import cp_model
# Clone the entire repo.
!git clone -l -s https://github.com/mswastik/optimization.git cloned-repo
%cd cloned-repo

Cloning into 'cloned-repo'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 149 (delta 57), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (149/149), 2.14 MiB | 2.44 MiB/s, done.
Resolving deltas: 100% (57/57), done.
/content/cloned-repo/cloned-repo


In [24]:
#Importing Data
df=pd.read_excel('orders1.xlsx')
ma=pd.read_excel('item master.xlsx')
bu=pd.read_excel('item master.xlsx',sheet_name='bulk')
pa=pd.read_excel('item master.xlsx',sheet_name='packing')

# Formating Data
df = df[df['Line']=='Tmmthpkhti Limi']
df = df[df['Plant']==1024]
del df['Line']
df['due'] = df['Dispatch Date'] - df['SO Date'].min()
df['due'] = df['due'].dt.days
df['sodate'] = df['SO Date'] - df['SO Date'].min()
df['sodate'] = df['sodate'].dt.days
df.reset_index(inplace=True,drop=True)
df['Key']=df['SO'].astype('str')+df['FG Code']
del df['Dispatch Date']
del df['SO Date']
del df['Description']
del df['Customer']
df['Order Qty'] = df[df['Order Qty']!=0]['Order Qty']
df['Order Qty'] = df['Order Qty']*1000
df=df[df['Key']!='17023287gd39000800iX']


# Creating Parameter Data
jobs =df['Key']
op =[1,2,3,4]  # Ignore as of now
mc = [1,2,3,4,5,6]
#horizon = max(df["due"])+900
due = df.set_index('Key')
dur=df[['Key','FG Code','Order Qty']]
dur=dur.merge(pa[["FG Code","Line","Output"]], on="FG Code")
dur['Output']=dur['Output']*100
dur['duration']=dur['Order Qty']/dur['Output']
dur['duration'] = dur['duration'].astype('float64').round(decimals = 2)
dur.set_index(['Key','Line'],inplace=True)
del dur['FG Code']
dur['Output'].replace(0,1,inplace=True)
df['Order Qty']=df['Order Qty'].fillna(0).astype('int')
dur['duration']=dur['duration'].fillna(0).astype('int')
dur['Output']=dur['Output'].fillna(0).astype('int')
horizon = max(dur['duration']*100)

In [25]:
#Testing
#dur['duration'].loc[('17023287gd29904300iX',1)]
#dur
setup = pd.DataFrame(columns=('Key1','Key2','setup'))
for i in df['Key']:
  for k in df['Key']:
    if df[df['Key']==i]["FG Code"].values[0]==df[df['Key']==k]["FG Code"].values[0]:
      f = {'Key1':[i],"Key2":[k],"setup":[0]}
    else:
      f = {'Key1':[i],"Key2":[k],"setup":[40]}
    f = pd.DataFrame(data=f)
    setup = pd.concat([setup,f])
setup.set_index(['Key1','Key2'],inplace=True)

In [26]:
model = cp_model.CpModel()
task_type = collections.namedtuple('task_type', 'start end run1')
mtj = collections.defaultdict(list)
all_tasks = {}

for j in jobs:
  for m in mc:
    suffix = '_%s_%i' % (j,m)
    start = model.NewIntVar(0, horizon, "start"+suffix)
    end = model.NewIntVar(0, horizon, "end"+suffix)
    run1 = model.NewIntVar(0, dur['duration'].loc[(j,m)].item(), "run"+suffix)
    #duration = model.NewIntervalVar(start, run1, end, "duration"+suffix)
    all_tasks[j, m] = task_type(start=start, end=end, run1=run1)
    #mtj[m].append(duration)
 
#for m in mc:
#  model.AddNoOverlap(mtj[m])
for i in jobs:
  for m in mc:
    model.Add(all_tasks[i, m].run1==all_tasks[i, m].end-all_tasks[i, m].start)


com = {}
for j in jobs:
  tt= 0
  for m in mc:
    comp = all_tasks[j,m].run1*dur["Output"].loc[(j,m)]
    tt = tt + comp
  com[j] = tt
  model.Add(com[j]*10 >= df[df["Key"]==j]['Order Qty'].values[0])


a={}
for i in jobs:
  for m in mc:
    if all_tasks[i,m].run1>0:
      for k in jobs:
        if k != i:
          a[i,k,m] = model.NewBoolVar('previous')
      
for i in jobs:
  for m in mc:
    if all_tasks[i,m].run1>0:
      for k in jobs:
        if k != i:
          model.AddBoolOr([a[i,k,m],a[k,i,m]])
          model.Add(all_tasks[k, m].start - all_tasks[i, m].end >= setup.loc[(i,k)].values[0]).OnlyEnforceIf(a[i,k,m])
          model.Add(all_tasks[i, m].start - all_tasks[k, m].end >= setup.loc[(i,k)].values[0]).OnlyEnforceIf(a[i,k,m].Not())

'''
com1 = {}
for j in df['Key']:
  tt= 0
  for m in mc:
    comp1 = (all_tasks[j,m].end - all_tasks[j,m].start)*dur["Output"].loc[(j,m)]
    tt = tt + comp1
  com1[j] = tt
  model.Add(com1[j]*10 <= df[df["Key"]==j]['Order Qty'].values[0]+500)
'''

ck = model.NewIntVar(-horizon, horizon, 'dummy')
model.AddMaxEquality(ck,[all_tasks[j,m].end for j in jobs for m in mc])

model.Minimize(ck)
solver = cp_model.CpSolver()
solver.Solve(model)
solver.StatusName()
#solver.ObjectiveValue()

'FEASIBLE'

In [30]:
ff=pd.DataFrame()
for m in mc:
  for j in jobs:
    kk={'Key':j,'line':m,'start':solver.Value(all_tasks[j,m].start),'run':solver.Value(all_tasks[j,m].run1),'end':solver.Value(all_tasks[j,m].end)}
    ff=ff.append(kk,ignore_index=True)

In [31]:
ff

,Key,end,line,run,start
0,17023287gd29904300iX,40.0,1.0,0.0,40.0
1,17023287gd29907800iX,80.0,1.0,0.0,80.0
2,17023287gd55102400iX,400.0,1.0,0.0,400.0
3,17023572gd43208500iX,320.0,1.0,0.0,320.0
4,17023636gd280043C0iX,692.0,1.0,124.0,568.0
...,...,...,...,...,...
97,17023439gd28311300iX,85.0,6.0,5.0,80.0
98,17023440gd23010000gR,125.0,6.0,0.0,125.0
99,4501763335gd23010000Ud,408.0,6.0,0.0,408.0
100,4501763335gd251100C0Ud,368.0,6.0,0.0,368.0


In [33]:
#Exploring Data
!pip install dtale
from dtale import show
import dtale.app as dtale_app
dtale_app.USE_COLAB = True
show(ff)

     |████████████████████████████████| 7.6MB 2.8MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 194kB 47.0MB/s 
     |████████████████████████████████| 1.8MB 32.7MB/s 
     |████████████████████████████████| 645kB 45.1MB/s 
     |████████████████████████████████| 952kB 40.6MB/s 
     |████████████████████████████████| 3.4MB 52.1MB/s 
     |████████████████████████████████| 194kB 38.8MB/s 
     |████████████████████████████████| 1.8MB 36.8MB/s 
     |████████████████████████████████| 358kB 49.0MB/s 
  Created wheel for dash: filename=dash-1.16.0-cp36-none-any.whl size=82327 sha256=5a5417ea4f4ed63960cade68034c80b912e4633c27a68b06df02f59c073a7c0e
  Stored in directory: /root/.cache/pip/wheels/5e/ee/63/1ba2ec6156da00e04457f46dd658a3f8c31247268eb62f93d0
  Created wheel for dash-colorscales: filename=dash_colorscales-0.0.4-cp36-none-any.whl size=62599 sha256=86aac125b0d7f804db8ce52e25c3

https://7ib8iv8151-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/1

In [ ]:
model.Minimize(dummy)
solver = cp_model.CpSolver()
solver.Solve(model)
solver.StatusName()

'INFEASIBLE'

In [ ]:
delay

SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(S

**Indices**  
j - job  
m - machine  

**Decision Variable**  
$s_{j,m}$ - start time of job j at machine m  
$e_{j,m}$ - end time of job j at machine m  
$d_{j,m}$ - duration of job j at machine m  

**Parameters**  
H - Horizon  
$d_j$ - delivery date of job j  
$q_j$ - order qty of job j  
$o_{j,m}$ - output per hour of job j at machine m  

**Objective**  
$ \sum_{j} d_j - max(e_{j,m})  \qquad \forall m \in mc$  

**Constraints**  
$ \sum_m e_{j,m} < s_{i,m} or s_{j,m} > e_{i,m} \qquad \forall j,i \in jobs$  
$ \sum_j o_{j,m} \times d_{j,m} >= q_j \qquad \forall m \in ma $
